In [ ]:
# !mkdir -p ~/aiffel/reuters_classifiaction

In [ ]:
from tensorflow.keras.datasets import reuters
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [ ]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=10000, test_split=0.2)

In [ ]:
print('훈련 샘플의 수: {}'.format(len(x_train)))
print('테스트 샘플의 수: {}'.format(len(x_test)))
print(x_train[0])
print(x_test[0])

In [ ]:
num_classes = max(y_train) + 1
print('클래스의 수 : {}'.format(num_classes))

In [ ]:
print('훈련용 뉴스의 최대 길이 :{}'.format(max(len(l) for l in x_train)))
print('훈련용 뉴스의 평균 길이 :{}'.format(sum(map(len, x_train))/len(x_train)))

plt.hist([len(s) for s in x_train], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
fig, axe = plt.subplots(ncols=1)
fig.set_size_inches(11,5)
sns.countplot(x=y_train)
plt.show()

In [ ]:
unique_elements, counts_elements = np.unique(y_train, return_counts=True)
print("각 클래스 빈도수:")
print(np.asarray(counts_elements))

In [ ]:
# decode 디버깅 해보기
word_index = reuters.get_word_index(path="reuters_word_index.json")
index_to_word = { index+3 : word for word, index in word_index.items() }
index_to_word[0] = "<pad>"
index_to_word[1] = "<sos>"
index_to_word[2] = "<unk>"

In [ ]:
def decode(encoded_list):
    return ' '.join([index_to_word[index] for index in encoded_list])

decode(x_train[0])

In [ ]:
decode([4, 12000, 23, 133, 6, 30, 515])

In [ ]:
x_train = [decode(x_train[i]) for i in range(len(x_train))]
print(len(x_train))
x_test = [decode(row) for row in x_test]
print(len(x_test))

In [ ]:
import random

k = 5
print(random.sample(x_train, k))
print(random.sample(x_test, k))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
dtmvector = CountVectorizer()
x_train_dtm = dtmvector.fit_transform(x_train)
print(x_train_dtm.shape)

In [ ]:
tfidf_transformer = TfidfTransformer()
tfidfv = tfidf_transformer.fit_transform(x_train_dtm)
print(tfidfv.shape)

In [ ]:
from sklearn.naive_bayes import MultinomialNB #다항분포 나이브 베이즈 모델
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score #정확도 계산

In [ ]:
model = MultinomialNB()
model.fit(tfidfv, y_train)

x_test_dtm = dtmvector.transform(x_test) #테스트 데이터를 DTM으로 변환
tfidfv_test = tfidf_transformer.transform(x_test_dtm) #DTM을 TF-IDF 행렬로 변환

predicted = model.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

In [ ]:
probability_3 = model.predict_proba(tfidfv_test[3])[0]

plt.rcParams["figure.figsize"] = (11,5)
plt.bar(model.classes_, probability_3)
plt.xlim(-1, 21)
plt.xticks(model.classes_)
plt.xlabel("Class")
plt.ylabel("Probability")
plt.show()

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
print(classification_report(y_test, model.predict(tfidfv_test), zero_division=0))

- macro: 단순평균
- weighted: 각 클래스에 속하는 표본의 개수로 가중평균
- accuracy: 정확도. 전체 학습 데이터의 개수에서 클래스를 정확하게 맞춘 개수의 비율

In [ ]:
def graph_confusion_matrix(model, x_test, y_test):  # , classes_name):
    df_cm = pd.DataFrame(
        confusion_matrix(y_test, model.predict(x_test))
    )  # , index=classes_name, columns=classes_name)
    fig = plt.figure(figsize=(12, 12))
    heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    heatmap.yaxis.set_ticklabels(
        heatmap.yaxis.get_ticklabels(), rotation=0, ha="right", fontsize=12
    )
    heatmap.xaxis.set_ticklabels(
        heatmap.xaxis.get_ticklabels(), rotation=45, ha="right", fontsize=12
    )
    plt.ylabel("label")
    plt.xlabel("predicted value")


graph_confusion_matrix(model, tfidfv_test, y_test)

## Complement Naive Bayes Classifier(CNB)
> 나이브 베이지안 분류기는 독립 변수가 '조건부로 독립적'이라는 가정을 하기 때문에, 문서가 특정 분류에 속할 실제 확률을 사용할 때 문제가 발생할 수 있습니다.

예를 들어 많은 샘플(sample)이 특정 클래스에 치우쳐져 있을 경우, 결정 경계의 가중치가 한쪽으로 치우쳐져 모델이 특정 클래스를 선호할 수 있습니다.

데이터가 불균형할 경우를 대비해 나이브 베이즈 분류기를 보완한 것이 컴플리먼트 나이브 베이즈 분류기입니다. 컴플리먼트 나이브 베이즈 분류기는 데이터의 불균형을 고려하여 가중치를 부여하는 특징을 가지고 있습니다. 따라서 컴플리먼트 나이브 베이즈 분류기는 나이브 베이즈 분류기(MultinomialNB)보다 성능이 일반적으로 더 좋습니다.


---
### 라벨 불균형이 미치는 영향 예시
나이브 베이지안 분류기의 결정 경계가 클래스 불균형 때문에 왜곡되는 현상을 수치적으로 이해하기 위해 간단한 예를 들어 설명하겠습니다.


우리가 두 개의 클래스 A와 B를 가진 이진 분류 문제를 다룬다고 가정해 봅시다. 여기서 클래스 A에 속하는 샘플이 90%, 클래스 B에 속하는 샘플이 10%라고 가정합니다. 또한, 단 하나의 특성 $(X)$를 가지고 있으며, 이 특성의 값은 0과 1 중 하나입니다.

#### 데이터 분포
- 클래스 A (90%): $[X=0]$ 50%, $[X=1]$ 50%
- 클래스 B (10%): $[X=0]$ 20%, $[X=1]$ 80%

즉, 클래스 A에 속하는 샘플이 훨씬 많고, 클래스 B에 속하는 샘플은 적습니다. 나이브 베이지안 분류기는 각 클래스의 사후 확률을 계산하여 가장 높은 사후 확률을 가진 클래스로 분류합니다.

### 나이브 베이지안 분류기 계산 과정

1. **사전 확률**:
    - $(P(A) = 0.9)$
    - $(P(B) = 0.1)$

2. **우도 (Likelihood)**:
    - $(P(X=0|A) = 0.5)$
    - $(P(X=1|A) = 0.5)$
    - $(P(X=0|B) = 0.2)$
    - $(P(X=1|B) = 0.8)$

3. **사후 확률 계산**:
    나이브 베이지안 분류기는 베이즈 정리를 사용하여 사후 확률을 계산합니다.
    
    $[
    P(A|X) = \frac{P(X|A) \cdot P(A)}{P(X)}
    ]$
    
    $[
    P(B|X) = \frac{P(X|B) \cdot P(B)}{P(X)}
    ]$

### 분류 예시

#### 케이스 1: $(X = 0)$ 일 때
- $(P(X=0) = P(X=0|A) \cdot P(A) + P(X=0|B) \cdot P(B) = 0.5 \cdot 0.9 + 0.2 \cdot 0.1 = 0.45 + 0.02 = 0.47)$

- $(P(A|X=0) = \frac{P(X=0|A) \cdot P(A)}{P(X=0)} = \frac{0.5 \cdot 0.9}{0.47} \approx 0.957)$

- $(P(B|X=0) = \frac{P(X=0|B) \cdot P(B)}{P(X=0)} = \frac{0.2 \cdot 0.1}{0.47} \approx 0.043)$

=> 이 경우, 나이브 베이지안 분류기는 클래스 A로 분류합니다.

#### 케이스 2: $(X = 1)$ 일 때
- $(P(X=1) = P(X=1|A) \cdot P(A) + P(X=1|B) \cdot P(B) = 0.5 \cdot 0.9 + 0.8 \cdot 0.1 = 0.45 + 0.08 = 0.53)$

- $(P(A|X=1) = \frac{P(X=1|A) \cdot P(A)}{P(X=1)} = \frac{0.5 \cdot 0.9}{0.53} \approx 0.849)$

- $(P(B|X=1) = \frac{P(X=1|B) \cdot P(B)}{P(X=1)} = \frac{0.8 \cdot 0.1}{0.53} \approx 0.151)$

=> 이 경우에도 나이브 베이지안 분류기는 클래스 A로 분류합니다.

### 문제점
이 예시에서 보듯이, 클래스 B의 샘플이 상대적으로 적기 때문에, 모델은 특성 값이 1인 경우에도 클래스 A로 분류하는 경향이 있습니다. 이는 클래스 불균형으로 인해 모델이 클래스 B를 과소평가하게 되는 예시입니다.

결과적으로, 실제로는 $(X=1)$일 때 클래스 B일 가능성이 높지만, 나이브 베이지안 분류기는 클래스 A로 잘못 분류하게 됩니다. 이로 인해 결정 경계가 왜곡되고, 클래스 B의 중요도를 무시하게 되는 문제가 발생합니다.

In [56]:
cb = ComplementNB()
cb.fit(tfidfv, y_train)

ComplementNB()

In [57]:
predicted = cb.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.7707034728406055


## Logistic Regression

In [ ]:
lr = LogisticRegression(C=10000, penalty='l2', max_iter=3000, verbose=True)
lr.fit(tfidfv, y_train)

In [ ]:
predicted = lr.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

## SVM
- descision boundary
- support vector: Decision Boundary와 가장 가까운 각 클래스의 데이터를 서포트 벡터라고 한다. Decision Boundary에 해당되는 벡터가 아니다.
- cost: descision boundary와 margin간의 간격 (margin과 반비례)

In [58]:
lsvc = LinearSVC(C=1000, penalty='l1', max_iter=3000, dual=False)
lsvc.fit(tfidfv, y_train)

/opt/conda/lib/python3.9/site-packages/sklearn/svm/_base.py:1199: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(C=1000, dual=False, max_iter=3000, penalty='l1')

In [59]:
prev = lsvc.predict(tfidfv_test)
print("정확도:", accuracy_score(y_test, prev)) #예측값과 실제값 비교

정확도: 0.7876224398931434


## Decision Tree

> 트리 계열의 모델들은 고차원이고 희소한 데이터에 대해서는 성능이 나오지 않는다는 특징이 있습니다. 

DTM이나 TF-IDF 행렬의 경우 고차원이면서 대부분의 값이 0인 희소한 데이터이므로 트리 계열의 모델보다는 선형 분류 모델을 통해 접근하는 것이 더 나은 접근일 수 있습니다.

In [54]:
tree = DecisionTreeClassifier(max_depth=10, random_state=0)
tree.fit(tfidfv, y_train)

DecisionTreeClassifier(max_depth=10, random_state=0)

In [55]:
predicted = tree.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.6202137132680321


## Random Forest


In [60]:
forest = RandomForestClassifier(n_estimators=5, random_state=0)
forest.fit(tfidfv, y_train)

RandomForestClassifier(n_estimators=5, random_state=0)

In [61]:
predicted = forest.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.674087266251113


## Gradient Boosting classifier

> 그래디언트 부스팅 트리는 일부 특성을 무시한다는 특징을 가지고 있습니다. 그래서 보통 랜덤 포레스트를 먼저 사용해보고, 성능이나 예측 시간 면에서 만족스럽지 않은 경우에 그래디언트 부스팅 트리를 시도해보는 것이 좋습니다.

일반적으로 1 ~ 5 정도의 깊지 않은 트리를 사용하므로 메모리도 적게 사용하고 예측도 빠릅니다.

In [62]:
# 15분 정도 소요될 수 있습니다.
grbt = GradientBoostingClassifier(random_state=0) # verbose=3
grbt.fit(tfidfv, y_train)

GradientBoostingClassifier(random_state=0)

In [63]:
predicted = grbt.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.7662511130899377


In [66]:
voting_classifier = VotingClassifier(estimators=[
        ('lr', lr), ('cb', cb), ('grbt', grbt)], voting='soft', verbose=True)
voting_classifier.fit(tfidfv, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =       444866     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.43889D+04    |proj g|=  2.96374D+03

At iterate   50    f=  1.30717D+03    |proj g|=  1.63867D+01

At iterate  100    f=  4.87098D+02    |proj g|=  1.84201D+00

At iterate  150    f=  4.62244D+02    |proj g|=  6.86419D-01

At iterate  200    f=  4.59886D+02    |proj g|=  6.36858D-01

At iterate  250    f=  4.59097D+02    |proj g|=  3.70866D-01

At iterate  300    f=  4.56964D+02    |proj g|=  1.24828D+00

At iterate  350    f=  4.55066D+02    |proj g|=  1.43875D-01

At iterate  400    f=  4.53239D+02    |proj g|=  3.17502D-01

At iterate  450    f=  4.51876D+02    |proj g|=  2.98869D-01

At iterate  500    f=  4.50581D+02    |proj g|=  1.51322D+00

At iterate  550    f=  4.49404D+02    |proj g|=  2.27625D-01

At iterate  600    f=  4.48576D+02    |proj g|=  2.10635D-01

At iterate  650    f=  4.4

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 10.8min finished


[Voting] ....................... (1 of 3) Processing lr, total=10.8min
[Voting] ....................... (2 of 3) Processing cb, total=   0.1s


KeyboardInterrupt: 

In [ ]:
predicted = voting_classifier.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교